In [1]:
from bertopic import BERTopic

import numpy as np
import pickle

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
from nltk.stem import PorterStemmer
# from textblob import TextBlob
from tqdm import tqdm
import re

In [3]:
df=pd.read_csv('pmodata_300k.csv')

In [4]:
data=df[df['initforward'].isna()==False]

data.shape

(147007, 27)

In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
stopwords1=set(stopwords.words('english'))
stopwords2=set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

all_stopwords=set(stopwords1|stopwords2)

lemmatizer=WordNetLemmatizer()
ps = PorterStemmer()
def decontract(sentence):
    sentence=re.sub(r"n\'t"," not",sentence)
    sentence=re.sub(r"\'re"," are",sentence)
    sentence=re.sub(r"\'s"," is",sentence)
    sentence=re.sub(r"\'d"," would",sentence)
    sentence=re.sub(r"\'ll"," will",sentence)
    sentence=re.sub(r"\'ve"," have",sentence)
    sentence=re.sub(r"\'m"," am",sentence) 
    return sentence

def preprocess(desc):
    desc=re.sub(r"http\S+","",desc)
    desc=decontract(desc)
    desc=re.sub("\S*\d\S*","",desc).strip()
    desc=re.sub('[^A-Za-z]+',' ',desc)
    desc=' '.join(m.lower() for m in word_tokenize(desc) if m.lower() not in all_stopwords)
    desc=' '.join(lemmatizer.lemmatize(m,pos='a') for m in word_tokenize(desc) if m not in all_stopwords and len(m)>2)
    return desc

In [7]:
data['description'] = data['description'].apply(preprocess)

In [ ]:
# Initialize the BERTopic model
model = BERTopic()

# Fit the model on the entire dataset to obtain embeddings
descriptions = data['description'].tolist()
topics, embeddings = model.fit_transform(descriptions)

# Store the embeddings of the complete dataset
stored_embeddings = embeddings.copy()

In [14]:
# Specify the filename for the saved file
filename = 'embeddings.pkl'

# Save the embeddings using pickle
with open(filename, 'wb') as file:
    pickle.dump(stored_embeddings, file)

print(f"Embeddings saved to {filename}")

Embeddings saved to embeddings.pkl


In [7]:
#type(data)
#data.iloc[12099]['regdate']
unique_values = data['initforward'].unique()
unique_values

array(['1. DEABD - Department of Financial Services (Banking Division)',
       '1. GOVAP - Government of Andhra Pradesh',
       '1. GOVUP - Government of Uttar Pradesh', ...,
       '1. DHRES - Department of Health Research, 2. GOVGJ - Government of Gujarat',
       '1. DOPAT - Department of Personnel and Training, 2. DOTEL - Department of Telecommunications',
       '1. DORVU - Department of Revenue, 2. DOTEL - Department of Telecommunications'],
      dtype=object)

In [ ]:
#data.columns
#data.iloc[10013]['initforward']
#import pandas as pd


import pandas as pd
import numpy as np

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based on the date range and initforward match
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get the indices of the filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data using valid indices
    obtained_embeddings = stored_embeddings[valid_indices]

    return obtained_embeddings

# User inputs
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
initforward_str = input("Enter the initforward string: ")

# Filter the DataFrame to remove rows where 'initforward' is NaN
data = df[df['initforward'].isna() == False]

# Get the embeddings
obtained_embeddings = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings)

In [60]:
# obtained_embeddings now contains the embeddings for the filtered data
#obtained_embeddings.shape
#from bertopic import BERTopic
# Check if the embeddings are 1D and reshape them to 2D if necessary
if len(obtained_embeddings.shape) == 1:
    obtained_embeddings = obtained_embeddings.reshape(-1, 1)

# Initialize a new BERTopic model
topic_model = BERTopic(nr_topics=10, calculate_probabilities=False, verbose=True)

# Create dummy documents with the same length as obtained_embeddings
dummy_documents = ["doc" + str(i) for i in range(len(obtained_embeddings))]

# Fit the BERTopic model on the dummy documents and obtained embeddings
topics, _ = topic_model.fit_transform(dummy_documents, obtained_embeddings)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(11)  # Get 11 as one topic might be -1 (outliers)

# Print the top 10 topics and their top 10 words
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        print(f"Topic {topic}: ", ", ".join(words))

2024-01-23 22:32:21,712 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-23 22:32:58,365 - BERTopic - Dimensionality - Completed ✓
2024-01-23 22:32:58,369 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-23 22:32:59,259 - BERTopic - Cluster - Completed ✓
2024-01-23 22:32:59,261 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-23 22:32:59,484 - BERTopic - Representation - Completed ✓
2024-01-23 22:32:59,486 - BERTopic - Topic reduction - Reducing number of topics
2024-01-23 22:32:59,647 - BERTopic - Topic reduction - Reduced number of topics from 183 to 10


Topic 0:  doc0, doc5771, doc5815, doc5818, doc5803, doc5825, doc5770, doc580, doc5799, doc5798
Topic 1:  doc1119, doc1393, doc7557, doc7512, doc6962, doc6848, doc615, doc5946, doc5664, doc5482
Topic 2:  doc3778, doc4680, doc7929, doc7746, doc7119, doc6820, doc653, doc6345, doc6281, doc5551
Topic 3:  doc3474, doc3799, doc7710, doc6926, doc6669, doc654, doc637, doc5781, doc5632, doc5353
Topic 4:  doc2824, doc2990, doc898, doc786, doc780, doc6726, doc6625, doc5911, doc5626, doc4846
Topic 5:  doc117, doc1282, doc6554, doc6507, doc6429, doc5851, doc5453, doc4879, doc4825, doc4261
Topic 6:  doc192, doc2379, doc7015, doc6748, doc6237, doc620, doc6061, doc553, doc4596, doc4469
Topic 7:  doc86, doc7588, doc7188, doc7157, doc6654, doc5614, doc5458, doc4987, doc4464, doc2755
Topic 8:  doc7897, doc6127, doc4494, doc4332, doc4294, doc4090, doc3761, doc3285, doc3208, doc1995


In [ ]:
from sklearn.cluster import KMeans

# Number of clusters (topics)
num_clusters = 10

# Fit KMeans to the embeddings
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(obtained_embeddings)

# Predict the clusters
clusters = kmeans.predict(obtained_embeddings)

In [ ]:
# Assign the cluster labels to the original data
data['Cluster'] = clusters

# Examine and interpret each cluster
for i in range(num_clusters):
    print(f"Cluster {i}:")
    cluster_data = data[data['Cluster'] == i]
    
    # Here you might want to print out some representative documents or
    # any other statistics that could help you understand what the cluster is about
    # For example:
    print(cluster_data['description'].head())  # Adjust based on your data structure
    
    # Add any additional analysis you find necessary
    print("\n")

In [90]:
stored_embeddings.shape
# Get the counts of unique values in the 'initforward' column
initforward_counts = data['initforward'].value_counts()

# Retrieve the top value (the one with the maximum count)
top_initforward = initforward_counts.idxmax()

# Print the value and its count
print(f"The 'initforward' value with the maximum data points is: '{top_initforward}'")
print(f"Count: {initforward_counts[top_initforward]}")


The 'initforward' value with the maximum data points is: '1. DEABD - Department of Financial Services (Banking Division)'
Count: 12979


In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


model = BERTopic(embedding_model=embedding_model)

# Fit the model 
descriptions = data['description'].tolist()
topics, embeddings = model.fit_transform(descriptions)

In [74]:
# # After fitting, you can access the topic embeddings
# topic_embeddings = model.topic_embeddings

# # Store the topic embeddings
# stored_embeddings = topic_embeddings.copy()
# Store the embeddings of the complete dataset
#stored_embeddings = embeddings.copy()

# # Calculate topic embeddings by averaging document embeddings per topic
# topic_embeddings = np.zeros((len(set(topics)), embeddings.shape[1]))

# for topic in set(topics):
#     if topic != -1:  # Exclude the outlier topic if present
#         indices = [i for i, t in enumerate(topics) if t == topic]
#         topic_embeddings[topic] = np.mean(embeddings[indices], axis=0)

# # Store the topic embeddings
# stored_embeddings = topic_embeddings.copy()

# # Check the shape of the topic embeddings
# print("Shape of topic embeddings:", stored_embeddings.shape)

# Ensure embeddings are in a 2D numpy array
if isinstance(embeddings, list):
    embeddings = np.array(embeddings)
elif len(embeddings.shape) == 1:
    embeddings = embeddings.reshape(-1, 1)

# Calculate topic embeddings by averaging document embeddings per topic
topic_embeddings = np.zeros((len(set(topics)), embeddings.shape[1]))

for topic in set(topics):
    if topic != -1:  # Exclude the outlier topic if present
        indices = [i for i, t in enumerate(topics) if t == topic]
        topic_embeddings[topic] = np.mean(embeddings[indices], axis=0)

# Store the topic embeddings
stored_topic_embeddings = topic_embeddings.copy()

# Check the shape of the topic embeddings
print("Shape of topic embeddings:", stored_topic_embeddings.shape)

print(embeddings[:5])  # Print the first 5 embeddings to inspect their structure
print("Shape of embeddings:", embeddings.shape)

Shape of topic embeddings: (10, 1)
[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]
Shape of embeddings: (147007, 1)


In [11]:


from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Get embeddings from the sentence-transformer model
embeddings = embedding_model.encode(data['description'].tolist(), show_progress_bar=True)


print("Embedding of the first document:", embeddings[0])
print("Shape of embeddings:", embeddings.shape)


Batches:   0%|          | 0/4594 [00:00<?, ?it/s]

Embedding of the first document: [ 3.33634801e-02  3.26836035e-02 -1.47670329e-01 -5.74986413e-02
 -1.50998291e-02  9.03493986e-02 -1.49062956e-02  3.87717523e-02
 -6.60005882e-02 -2.33104248e-02  4.36904095e-02 -3.26045975e-02
  1.02416873e-02 -3.21899168e-02  3.10279876e-02  4.64709140e-02
 -2.35015675e-02  5.66720869e-03 -1.33569585e-02 -1.71295926e-02
 -6.29320815e-02 -5.44602610e-03 -4.39039879e-02 -7.34623969e-02
  6.01073168e-02  1.06643289e-02 -6.87981769e-03 -3.61237861e-02
  8.59590471e-02  2.41204724e-02  8.99317563e-02  3.37226968e-03
 -8.59604254e-02  9.26114339e-03  5.33274375e-02 -5.64609468e-03
 -5.13846204e-02  4.01779637e-02  1.33804064e-02 -3.41936275e-02
  6.69373497e-02 -9.23117995e-03 -3.76868956e-02 -5.05258106e-02
  8.15688893e-02 -6.59956015e-04 -1.52508682e-02  3.76336537e-02
 -2.72375327e-02 -9.93224513e-03 -3.60711291e-02 -1.07922591e-01
  3.27603593e-02  2.42191833e-02 -2.83576902e-02 -2.51412056e-02
  1.07735604e-01  2.52739526e-02 -7.39892013e-03 -5.91449

In [12]:
# Specify the filename for the saved file
filename = 'final_embedding.pkl'

# Save the embeddings using pickle
with open(filename, 'wb') as file:
    pickle.dump(embeddings, file)

print(f"Embeddings saved to {filename}")

Embeddings saved to final_embedding.pkl


In [8]:
import pickle

# Specify the filename for the file to be loaded
filename = 'final_embedding.pkl'

# Load the embeddings from the file
with open(filename, 'rb') as file:
    stored_embeddings = pickle.load(file)

print("Embeddings loaded successfully.")
stored_embeddings.shape


Embeddings loaded successfully.


(147007, 384)

In [130]:
import numpy as np

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get indices of filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data 
    obtained_embeddings = stored_embeddings[valid_indices]

    return obtained_embeddings


starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
initforward_str = input("Enter the initforward string: ")


data = df[df['initforward'].isna() == False]

# Get the embeddings
obtained_embeddings = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, embeddings)

Enter the starting date (YYYY-MM-DD):  2021-01-01
Enter the ending date (YYYY-MM-DD):  2021-12-31
Enter the initforward string:  1. DEABD - Department of Financial Services (Banking Division)


In [10]:
import pandas as pd


data['regdate'] = pd.to_datetime(data['regdate'])


filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                     (data['regdate'] <= pd.to_datetime(ending_date)) &
                     (data['initforward'] == initforward_str)]

# GMM 

In [96]:
from sklearn.mixture import GaussianMixture

# Number of topics
n_components = 10

# Apply GMM to the obtained embeddings
gmm = GaussianMixture(n_components=n_components, random_state=42)
gmm.fit(obtained_embeddings)
topic_probabilities = gmm.predict_proba(obtained_embeddings)

# Assign the most likely topic to each document in the filtered dataset
filtered_data['Topic'] = topic_probabilities.argmax(axis=1)

from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

# Print top words for each topic in the filtered data
for i in range(n_components):
    topic_documents = filtered_data[filtered_data['Topic'] == i]['description']
    top_words = get_top_words(topic_documents, 10)
    print(f"Topic {i}: {', '.join([word for word, freq in top_words])}")

In [111]:
# GMM + LDA

In [114]:
from sklearn.mixture import GaussianMixture

# Apply GMM to the obtained embeddings
gmm = GaussianMixture(n_components=10, random_state=42)
gmm.fit(obtained_embeddings)
clusters = gmm.predict(obtained_embeddings)

# Assign cluster labels to the filtered data
filtered_data['Cluster'] = clusters

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

n_topics_per_cluster = 1  # Number of subtopics you want in each cluster

for cluster in range(10):
    # Filter the documents for each cluster
    cluster_docs = filtered_data[filtered_data['Cluster'] == cluster]['description']

    # Vectorize the documents
    vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2)
    doc_term_matrix = vectorizer.fit_transform(cluster_docs)

    # Apply LDA
    lda = LatentDirichletAllocation(n_components=n_topics_per_cluster, random_state=42)
    lda.fit(doc_term_matrix)

    # Display the top words in each subtopic
    print(f"\nCluster {cluster}:")
    for idx, topic in enumerate(lda.components_):
        print(f"Topic {idx}: ", [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])


Cluster 0:
Topic 0:  ['india', 'rs', '39', 'branch', 'sbi', '2020', 'account', 'sir', 'loan', 'bank']

Cluster 1:
Topic 0:  ['request', 'rs', '39', 'sbi', 'branch', '2020', 'account', 'sir', 'loan', 'bank']

Cluster 2:
Topic 0:  ['request', 'branch', 'rs', '2020', '39', 'account', '160', 'sir', 'loan', 'bank']

Cluster 3:
Topic 0:  ['39', 'branch', 'card', '160', 'account', '2020', 'sir', 'loan', 'rs', 'bank']

Cluster 4:
Topic 0:  ['2020', 'request', 'branch', 'india', '39', '160', 'account', 'loan', 'sir', 'bank']

Cluster 5:
Topic 0:  ['hai', 'india', 'branch', '2020', 'rs', '39', 'account', 'sir', 'loan', 'bank']

Cluster 6:
Topic 0:  ['money', '39', 'rs', 'branch', '2020', '160', 'account', 'loan', 'sir', 'bank']

Cluster 7:
Topic 0:  ['help', '2020', 'request', 'branch', 'rs', '39', 'account', 'sir', 'loan', 'bank']

Cluster 8:
Topic 0:  ['india', 'sbi', 'branch', '39', 'rs', 'account', '2020', 'sir', 'loan', 'bank']

Cluster 9:
Topic 0:  ['india', 'rs', 'sbi', '2020', '39', 'br

In [91]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

num_topics_per_cluster = 1  # Number of topics in each cluster
num_words_per_topic = 10     # Number of words per topic

# Process each cluster
for cluster_num in range(10):  # Assuming 10 clusters from GMM
    # Filter documents in the current cluster
    cluster_documents = filtered_data[filtered_data['Cluster'] == cluster_num]['description']

    # Create a TF-IDF representation of the documents
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
    tfidf = vectorizer.fit_transform(cluster_documents)

    # Apply NMF
    nmf = NMF(n_components=num_topics_per_cluster, random_state=42)
    nmf.fit(tfidf)

    # Display the top words for each topic
    print(f"Cluster {cluster_num} Topics:")
    for topic_idx, topic in enumerate(nmf.components_):
        top_features_idx = topic.argsort()[-num_words_per_topic:]
        top_features = [vectorizer.get_feature_names_out()[i] for i in top_features_idx]
        top_features.reverse()  # To get the top words in descending order of weight
        print(f"Topic {topic_idx}: {', '.join(top_features)}")
    print("\n")

Cluster 0 Topics:
Topic 0: bank, loan, sir, account, sbi, 39, 2020, branch, rs, help


Cluster 1 Topics:
Topic 0: bank, loan, sir, account, branch, 39, 2020, sbi, rs, home


Cluster 2 Topics:
Topic 0: bank, loan, sir, account, branch, 39, 2020, help, rs, sbi


Cluster 3 Topics:
Topic 0: bank, rs, loan, account, 2020, card, credit, branch, sbi, charges


Cluster 4 Topics:
Topic 0: bank, loan, sir, account, 39, request, pay, help, india, sbi


Cluster 5 Topics:
Topic 0: bank, loan, account, sir, 39, branch, rs, 2020, money, help


Cluster 6 Topics:
Topic 0: bank, loan, sir, account, branch, money, 39, rs, 2020, help


Cluster 7 Topics:
Topic 0: bank, loan, sir, account, 39, branch, rs, help, pay, request


Cluster 8 Topics:
Topic 0: bank, loan, sir, account, 39, rs, help, branch, 2020, sbi


Cluster 9 Topics:
Topic 0: bank, loan, sir, 39, account, branch, sbi, 2020, rs, help




In [115]:
from sklearn.cluster import KMeans

num_clusters = 10

# Apply KMeans 
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(obtained_embeddings)

# Assign the cluster labels to data
filtered_data['Topic'] = clusters
for i in range(num_clusters):
    print(f"Documents in Topic {i}:")
    # Retrieve documents in the cluster
    cluster_docs = filtered_data[filtered_data['Topic'] == i]['description']
    
   
    print(cluster_docs.head())  

Documents in Topic 0:
772     MR. MODI, PM INDIA,THE GRIEVANCE PMOPG/E/2020/...
1062    Dear TeamGreetings for the day.... --- Clarifi...
1065    Sir please i request you to take strict action...
1144    Sir, I A Nirmala am requesting you that I have...
1413    RESPECTED SIR I AM FARMAR AT VILLAGE TIWARIPUR...
Name: description, dtype: object
Documents in Topic 1:
101    Extension of cut of date from 1st April 2020 t...
194    Dear sir I closed my NSS account. after 20% de...
354    Pension scheme introduced in 1993 but around 5...
365    Respected Sir,I am approaching your office for...
495    INSURANCE COMPANY WAS NOT FOLLOW THE IRDA RULE...
Name: description, dtype: object
Documents in Topic 2:
13     Ref.No.2020-21/L/12/0306 TO PMORef :- a) PMO C...
105    I have taken home loan from Central Bank of In...
121    The Axis Bank Ganesh Chandra Avenue Branch is ...
188    Hon&#39;ble Prime Minister ModijiWrites to kno...
200    Sir, My earlier letter Ref: Your Registration ...
Name: de

In [127]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

# Print top words for each topic
for i in range(num_clusters):
    print(f"Top words in Topic {i}:")
    cluster_docs = filtered_data[filtered_data['Topic'] == i]['description']
    top_words = get_top_words(cluster_docs, 10)
    print(", ".join([word for word, freq in top_words]))





Top words in Topic 0:
bank, loan, sir, account, sbi, rs, branch, 39, 2020, request
Top words in Topic 1:
bank, loan, sir, account, branch, sbi, 2020, 39, 160, rs
Top words in Topic 2:
bank, loan, sir, account, 2020, rs, branch, 39, india, help
Top words in Topic 3:
bank, loan, sir, account, 2020, sbi, branch, 39, rs, india
Top words in Topic 4:
bank, loan, sir, account, rs, 39, branch, request, 2020, help
Top words in Topic 5:
bank, loan, sir, account, 39, rs, 2020, branch, hai, india
Top words in Topic 6:
bank, loan, sir, account, 39, 2020, sbi, branch, rs, india
Top words in Topic 7:
bank, loan, sir, account, 2020, 39, branch, rs, india, request
Top words in Topic 8:
bank, loan, sir, account, 2020, 39, 160, branch, rs, help
Top words in Topic 9:
bank, rs, loan, sir, 2020, account, 160, card, branch, 39


In [117]:
num_clusters = 10 

for i in range(num_clusters):
    
    #print(f"Documents in Topic {i}:")
    cluster_docs = filtered_data[filtered_data['Topic'] == i]['description']
    
    #print(cluster_docs.head())  # Display some documents for each topic

    print(f"Top words in Topic {i}:")
    top_words = get_top_words(cluster_docs, 10)  # Get top 10 words
    print(", ".join([word for word, freq in top_words]))
    print("\n")

Top words in Topic 0:
bank, loan, sir, account, sbi, rs, branch, 39, 2020, request


Top words in Topic 1:
bank, loan, sir, account, branch, sbi, 2020, 39, 160, rs


Top words in Topic 2:
bank, loan, sir, account, 2020, rs, branch, 39, india, help


Top words in Topic 3:
bank, loan, sir, account, 2020, sbi, branch, 39, rs, india


Top words in Topic 4:
bank, loan, sir, account, rs, 39, branch, request, 2020, help


Top words in Topic 5:
bank, loan, sir, account, 39, rs, 2020, branch, hai, india


Top words in Topic 6:
bank, loan, sir, account, 39, 2020, sbi, branch, rs, india


Top words in Topic 7:
bank, loan, sir, account, 2020, 39, branch, rs, india, request


Top words in Topic 8:
bank, loan, sir, account, 2020, 39, 160, branch, rs, help


Top words in Topic 9:
bank, rs, loan, sir, 2020, account, 160, card, branch, 39




# NMF

In [110]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF


filtered_data = data[data['initforward'] == '1. DEABD - Department of Financial Services (Banking Division)']

# Convert descriptions to a TF-IDF matrix
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = vectorizer.fit_transform(filtered_data['description'])

# Apply NMF
num_topics = 10  # Number of topics
nmf = NMF(n_components=num_topics, random_state=0)
nmf.fit(tfidf)

# Display the topics
num_top_words = 10
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic {topic_idx}: ", end='')
    top_features_idx = topic.argsort()[-num_top_words:][::-1]
    top_features = [vectorizer.get_feature_names_out()[i] for i in top_features_idx]
    print(", ".join(top_features))
    

Topic 0: 39, sir, pay, help, emi, business, situation, banks, covid, people
Topic 1: enclosures, scanned, shortly, initiated, fully, receipt, pg, copy, hard, sent
Topic 2: hai, se, ki, ke, ka, nahi, mai, mera, bhi, ko
Topic 3: salaries, salary, settlement, iba, bank, arrears, bipartite, paid, unions, board
Topic 4: bank, account, branch, complaint, rs, number, 2021, money, india, transaction
Topic 5: subsidy, pmay, home, application, loan, id, housing, scheme, received, clss
Topic 6: 160, dhfl, details, permanent, payment, madam, like, housing, process, quot
Topic 7: card, credit, rs, charges, sbi, payment, hdfc, outstanding, debit, pay
Topic 8: pension, sbi, branch, cppc, 2020, 2021, ppo, quot, da, office
Topic 9: loan, education, bank, mudra, home, manager, sir, branch, applied, help


In [118]:
import hdbscan

# Apply HDBSCAN to the obtained embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, prediction_data=True)
clusters = clusterer.fit_predict(obtained_embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]

# Assign cluster labels to the filtered clustered data
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

for cluster_num in set(clusters) - {-1}:  # Exclude noise (-1)
    # print(f"Documents in Cluster {cluster_num}:")
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    # print(cluster_docs.head())  # Display a few documents from the cluster
    
    # Get and print the top words for each cluster
    top_words = get_top_words(cluster_docs, 10)
    print(f"Top words in Cluster {cluster_num}: {', '.join([word for word, freq in top_words])}")
    print("\n")

Top words in Cluster 0: sbi, dhfl, life, fd, holders, bank, ibc, sir, resolution, process


Top words in Cluster 1: bank, 160, loan, rs, 2020, 39, sir, branch, passbook, charges


Top words in Cluster 2: senior, citizens, saab, 2020, credit, card, mera, pmopg, bank, quot


Top words in Cluster 3: bank, rs, loan, branch, 2020, account, charges, baroda, sbi, complain


Top words in Cluster 4: card, credit, sir, bank, 39, kindly, loan, active, charges, apply


Top words in Cluster 5: sir, bank, customer, risk, customers, mai, rbi, banks, payment, help


Top words in Cluster 6: bank, account, branch, credit, loan, request, 39, rs, customers, pay


Top words in Cluster 7: rs, fd, dhfl, transaction, bank, public, holders, money, account, charges


Top words in Cluster 8: loan, sir, bank, hai, taken, business, help, hdfc, want, time


Top words in Cluster 9: loan, bank, 39, education, sir, sbi, account, manager, da, banks


Top words in Cluster 10: bank, loan, account, 39, subsidy, online, co

In [119]:
# HDP

In [ ]:
from gensim import corpora
from gensim.models import HdpModel
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords

# Ensure you have the necessary NLTK data
nltk.download('stopwords')

# Preprocess the text data
def preprocess(text):
    result = []
    for token in simple_preprocess(text):
        if token not in stopwords.words('english'):
            result.append(token)
    return result

# Assuming 'data' is your DataFrame and 'description' is the column with text data
processed_docs = data['description'].map(preprocess)

# Create a dictionary and corpus needed for topic modeling
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Apply HDP model
hdp_model = HdpModel(corpus, dictionary)

# Display the topics
for i, topic in enumerate(hdp_model.print_topics(num_topics=10, num_words=10)):
    print(f"Topic {i}: {topic}")

In [ ]:
for i, topic_info in enumerate(hdp_model.print_topics(num_topics=10, num_words=10)):
    # The topic_info is a tuple of topic number and the weighted words
    # We split the string to extract only words
    words = topic_info[1].split("+")
    
    # Extract and clean each word
    topic_words = [word.split("*")[1].strip() for word in words]
    
    # Combine the words back into a single string
    print(f"Topic {i}: {', '.join(topic_words)}")

In [145]:
# topic_model = BERTopic()
# docs = filtered_data['description'].tolist()
# topics, probs = topic_model.fit_transform(docs, obtained_embeddings)
# len(filtered_data['description'].tolist())
import numpy as np
import pandas as pd

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based on the conditions
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get indices of filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data
    obtained_embeddings = np.array(stored_embeddings)[valid_indices]

    # Retrieve descriptions for the filtered data
    descriptions = filtered_data.loc[valid_indices, 'description'].tolist()

    return obtained_embeddings, descriptions

# Inputs from user
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
initforward_str = input("Enter the initforward string: ")

# Assuming 'data' is your DataFrame and 'embeddings' is the list/array of stored embeddings
data = df[df['initforward'].isna() == False]  # Filter data where 'initforward' is not NaN

# Get the embeddings and descriptions
obtained_embeddings, descriptions = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, embeddings)

# Now `obtained_embeddings` will contain the embeddings and `descriptions` will contain the corresponding descriptions


Enter the starting date (YYYY-MM-DD):  2021-01-01
Enter the ending date (YYYY-MM-DD):  2021-12-31
Enter the initforward string:  1. DEABD - Department of Financial Services (Banking Division)


In [ ]:
#type(descriptions)
topic_model = BERTopic()
docs = descriptions
topics, probs = topic_model.fit_transform(docs, obtained_embeddings)

In [150]:
# len(filtered_data['description'].tolist())
topic_info = topic_model.get_topic_info()

# Display the topics
print(topic_info)

     Topic  Count                                   Name  \
0       -1   2555                       -1_to_the_of_and   
1        0    983                        0_the_and_to_of   
2        1    293                        1_the_of_to_and   
3        2    277                         2_the_to_is_of   
4        3    258                         3_to_of_and_in   
..     ...    ...                                    ...   
103    102     11   102_rs_anandnagar_complain_ahmedabad   
104    103     11        103_autodebit_ssc_payment_pmsby   
105    104     11    104_bachchon_excess_remarks_manager   
106    105     10           105_nai_muje_hai_outstanding   
107    106     10  106_psb39s_bavjut_scholarship_barauni   

                                        Representation  \
0       [to, the, of, and, my, in, bank, is, for, not]   
1      [the, and, to, of, bank, my, is, for, in, this]   
2      [the, of, to, and, for, is, my, in, have, bank]   
3        [the, to, is, of, and, my, in, for, me

In [155]:
#bu



topic_model = BERTopic()
docs = descriptions
# Fit the BERTopic model on the dummy documents and obtained embeddings
topics, _ = topic_model.fit_transform(docs, obtained_embeddings)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(11)  # Get 11 as one topic might be -1 (outliers)

# Print the top 10 topics and their top 10 words
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        print(f"Topic {topic}: ", ", ".join(words))
        

#1. DORVU - Department of Revenue
        

Topic 0:  the, and, to, bank, of, my, is, in, for, this
Topic 1:  the, to, of, is, and, my, in, for, me, bank
Topic 2:  of, the, to, and, for, is, in, bank, my, have
Topic 3:  the, my, to, and, bank, of, not, is, me, have
Topic 4:  the, to, of, and, for, that, in, my, he, is
Topic 5:  and, the, in, to, of, we, for, is, my, will
Topic 6:  160, and, the, to, we, of, in, are, my, for
Topic 7:  of, to, and, the, my, in, for, loan, this, bank
Topic 8:  bank, of, the, and, in, for, my, to, is, this
Topic 9:  the, to, and, you, my, of, that, bank, in, is


In [156]:

topic_model = BERTopic()
docs = descriptions

topics, _ = topic_model.fit_transform(docs)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(11)  # Get 11 as one topic might be -1 (outliers)

# Print the top 10 topics and their top 10 words
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        print(f"Topic {topic}: ", ", ".join(words))

    

Topic 0:  hai, se, ki, ke, ka, ko, nahi, mai, mera, bhi
Topic 1:  subsidy, pmay, home, loan, housing, application, for, scheme, id, nhb
Topic 2:  account, branch, bank, of, the, is, to, open, not, address
Topic 3:  atm, transaction, amount, account, money, but, my, no, bank, debited
Topic 4:  mudra, business, loan, start, for, to, am, project, sir, my
Topic 5:  banks, privatisation, sector, public, are, private, banking, of, privatization, will
Topic 6:  pension, cppc, ppo, of, delhi, no, sbi, from, the, family
Topic 7:  loan, my, pay, family, am, help, me, job, to, amount
Topic 8:  pmegp, loan, project, under, the, for, application, manager, applied, unit
Topic 9:  education, 160, college, loan, my, university, fees, father, for, studies


In [157]:
#1. DORVU - Department of Revenue
# topic_model = BERTopic()
# docs = filtered_data['description'].tolist()
# topics, probs = topic_model.fit_transform(docs, obtained_embeddings)
# len(filtered_data['description'].tolist())
import numpy as np
import pandas as pd

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based on the conditions
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get indices of filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data
    obtained_embeddings = np.array(stored_embeddings)[valid_indices]

    # Retrieve descriptions for the filtered data
    descriptions = filtered_data.loc[valid_indices, 'description'].tolist()

    return obtained_embeddings, descriptions

# Inputs from user
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
initforward_str = input("Enter the initforward string: ")

# Assuming 'data' is your DataFrame and 'embeddings' is the list/array of stored embeddings
data = df[df['initforward'].isna() == False]  # Filter data where 'initforward' is not NaN

# Get the embeddings and descriptions
obtained_embeddings, descriptions = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, embeddings)

# Now `obtained_embeddings` will contain the embeddings and `descriptions` will contain the corresponding descriptions




Enter the starting date (YYYY-MM-DD):  2021-01-01
Enter the ending date (YYYY-MM-DD):  2021-12-31
Enter the initforward string:  1. DORVU - Department of Revenue


In [160]:

topic_model = BERTopic()
docs = descriptions

topics, _ = topic_model.fit_transform(docs)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(26)  


for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        print(f"Topic {topic}: ", ", ".join(words))

    

Topic 0:  the, of, and, to, in, is, refund, tax, income, my
Topic 1:  gst, to, we, the, and, my, is, for, our, not
Topic 2:  pan, card, my, to, the, name, application, is, in, it
Topic 3:  due, to, and, the, for, of, are, we, in, date
Topic 4:  and, customs, to, the, is, have, in, we, india, for
Topic 5:  tax, to, the, of, and, for, in, are, is, will
Topic 6:  my, the, to, pension, of, and, that, been, on, has
Topic 7:  hai, ki, se, ke, ka, ko, nahi, mera, bhi, hu
Topic 8:  customs, commissioner, of, suspension, tribunal, dated, principal, honble, had, central
Topic 9:  scanned, shortly, enclosures, initiated, fully, upon, receipt, hard, along, copy
Topic 10:  gst, the, to, of, is, tax, are, and, or, in
Topic 11:  gst, be, will, and, to, of, the, in, if, steel
Topic 12:  adhar, date, entire, site, extended, for, daythe, linking, same, reason
Topic 13:  tds, society, 201516, deposited, my, sahara, deducted, vide, 201819, cooperative
Topic 14:  fraud, gst, and, my, the, to, no, this, pan

In [161]:
#1. DORVU - Department of Revenue
# topic_model = BERTopic()
# docs = filtered_data['description'].tolist()
# topics, probs = topic_model.fit_transform(docs, obtained_embeddings)
# len(filtered_data['description'].tolist())
topic_model = BERTopic()
docs = descriptions
# Fit the BERTopic model on the dummy documents and obtained embeddings
topics, _ = topic_model.fit_transform(docs, obtained_embeddings)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(26)  # Get 11 as one topic might be -1 (outliers)

# Print the top 10 topics and their top 10 words
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        print(f"Topic {topic}: ", ", ".join(words))

Topic 0:  the, of, to, and, in, for, is, tax, on, my
Topic 1:  the, of, to, and, is, for, in, my, tax, not
Topic 2:  to, the, and, of, is, in, for, my, we, are
Topic 3:  the, to, and, of, my, in, is, as, not, on
Topic 4:  the, to, and, of, in, is, for, my, this, have
Topic 5:  the, to, of, and, for, in, is, on, my, have
Topic 6:  of, to, the, and, my, in, is, for, on, as
Topic 7:  the, for, to, of, and, is, we, in, that, gst
Topic 8:  to, the, and, of, is, for, in, on, are, my
Topic 9:  to, the, for, not, and, of, is, we, my, in
Topic 10:  the, to, not, and, in, my, of, is, for, tax
Topic 11:  of, the, osho, to, by, is, in, and, that, for
Topic 12:  of, and, to, the, in, is, as, not, by, for
Topic 13:  the, and, of, to, in, tax, this, income, is, for
Topic 14:  to, and, the, my, of, is, on, in, tax, for
Topic 15:  of, in, the, to, his, and, my, family, have, tax
Topic 16:  the, is, to, of, and, refund, on, it, insurance, in
Topic 17:  the, my, gramalaya, in, for, is, to, of, and, sanit

In [9]:

import numpy as np
import pandas as pd

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based on the conditions
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get indices of filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data
    obtained_embeddings = np.array(stored_embeddings)[valid_indices]

    # Retrieve descriptions for the filtered data
    descriptions = filtered_data.loc[valid_indices, 'description'].tolist()
    filtered_data = filtered_data.loc[valid_indices]

    return obtained_embeddings, descriptions, filtered_data

# Inputs from user
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
initforward_str = input("Enter the initforward string: ")

# Assuming 'data' is your DataFrame and 'embeddings' is the list/array of stored embeddings
data = df[df['initforward'].isna() == False]  # Filter data where 'initforward' is not NaN

# Get the embeddings and descriptions
obtained_embeddings, descriptions, filtered_data = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings)



Enter the starting date (YYYY-MM-DD):  2021-01-01
Enter the ending date (YYYY-MM-DD):  2021-12-31
Enter the initforward string:  1. DORVU - Department of Revenue


In [32]:
import hdbscan

# Apply HDBSCAN to the obtained embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, prediction_data=True)
clusters = clusterer.fit_predict(obtained_embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]

# Assign cluster labels to the filtered clustered data
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

for cluster_num in set(clusters) - {-1}:  # Exclude noise (-1)
    # print(f"Documents in Cluster {cluster_num}:")
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    # print(cluster_docs.head())  # Display a few documents from the cluster
    
    # Get and print the top words for each cluster
    top_words = get_top_words(cluster_docs, 10)
    print(f"Top words in Cluster {cluster_num}: {', '.join([word for word, freq in top_words])}")
    print("\n")

Top words in Cluster 0: sir, gst, gramalaya, refund, government, india, 39, rs, department, sanitation


Top words in Cluster 1: tax, office, order, income, kodaikannal, faa, cpio, assistant, commissioner, information


Top words in Cluster 2: tax, 39, pay, gst, currency, husband, pf, notice, rs, flat


Top words in Cluster 3: officers, india, tax, sir, review, dpc, huge, income, gst, 39


Top words in Cluster 4: tax, gst, business, kothur, account, money, number, refund, aadhar, district


Top words in Cluster 5: tax, 03, delhi, income, new, aforesaid, number, gst, department, date


Top words in Cluster 6: tax, income, years, rs, 2019, 2020, refund, dated, speed, year


Top words in Cluster 7: refund, tax, grievance, insurance, income, gst, quot, department, year, government


Top words in Cluster 8: tax, card, gst, sir, time, rs, pan, request, 39, income


Top words in Cluster 9: tax, income, rs, sir, 2020, gst, refund, 2021, india, cbic


Top words in Cluster 10: 160, gst, tax, 202

In [14]:
import pandas as pd
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# Assuming 'filtered_data' is a DataFrame that includes a 'preprocessed' column containing the text data
# Assuming 'obtained_embeddings' are your precomputed embeddings for the 'filtered_data'

def get_top_words(documents, n_top_words=10):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

def recursive_clustering(level, data, embeddings, min_cluster_size=15, min_samples=None, results=None):
    if results is None:
        results = {}

    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, prediction_data=True)
    clusters = clusterer.fit_predict(embeddings)

    unique_clusters = set(clusters) - {-1}  # Exclude noise if present

    for cluster_id in unique_clusters:
        cluster_mask = clusters == cluster_id
        cluster_data = data.iloc[cluster_mask]
        cluster_embeddings = embeddings[cluster_mask]

        # Extract top words for the current cluster
        documents = cluster_data['description'].tolist()
        top_words = get_top_words(documents)

        # Store top words with their frequencies in the results dictionary
        cluster_label = f"{level}{cluster_id}" if level else str(cluster_id)
        results[cluster_label] = {'top_words': [word for word, freq in top_words]}

        # Recursive call if the cluster is large enough
        if len(cluster_data) > 100:  # Adjust threshold as needed
            new_level = f"{cluster_label}."
            recursive_clustering(new_level, cluster_data, cluster_embeddings, min_cluster_size, min_samples, results)

    return results


hierarchical_topics = recursive_clustering("", filtered_data, obtained_embeddings)


In [15]:


# Function to display the hierarchical topics
def display_topics(hierarchical_topics, level=0):
    for cluster_label, info in hierarchical_topics.items():
        indent = '  ' * level
        # Since top_words is a list of words, directly join them
        top_words = ', '.join(info['top_words'])
        print(f"{indent}Cluster {cluster_label} Topics: {top_words}")

# Call the function to display the topics
display_topics(hierarchical_topics)



Cluster 0 Topics: tax, income, gst, sir, 2020, 39, pan, refund, rs, department
Cluster 0.0 Topics: tax, income, gst, sir, 2020, department, 39, refund, request, pan
Cluster 0.1 Topics: tax, gst, rs, income, sir, india, refund, 2021, demand, pan
Cluster 1 Topics: tax, gst, sir, 39, income, department, years, refund, request, government


In [12]:
def recursive_clustering(level, data, embeddings, min_cluster_size=15, min_samples=None, results=None):
    if results is None:
        results = {}

    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, prediction_data=True)
    clusters = clusterer.fit_predict(embeddings)

unique_clusters = set(clusters) - {-1}  # Exclude noise if present
    #unique_clusters = set(clusters)

    for cluster_id in unique_clusters:
        cluster_mask = clusters == cluster_id
        cluster_data = data.iloc[cluster_mask]
        cluster_embeddings = embeddings[cluster_mask]

        # Extract top words for the current cluster
        documents = cluster_data['description'].tolist()
        top_words = get_top_words(documents)

        # Store top words, their frequencies, and the number of documents in the cluster
        cluster_label = f"{level}{cluster_id}" if level else str(cluster_id)
        results[cluster_label] = {
            'top_words': [word for word, _ in top_words],
            'no_of_documents': len(cluster_data)
        }

        # Recursive call if the cluster is large enough
        if len(cluster_data) > 100:  # Adjust threshold as needed
            new_level = f"{cluster_label}."
            recursive_clustering(new_level, cluster_data, cluster_embeddings, min_cluster_size, min_samples, results)

    return results
# Assuming 'filtered_data' and 'obtained_embeddings' are defined as per your setup
hierarchical_topics = recursive_clustering("", filtered_data, obtained_embeddings)
#display_topics(hierarchical_topics)



In [13]:


def display_topics(hierarchical_topics, level=0):
    for cluster_label, info in hierarchical_topics.items():
        indent = '  ' * level
        top_words = ', '.join(info['top_words'])
        no_of_documents = info['no_of_documents']
        print(f"{indent}Cluster {cluster_label} (Documents: {no_of_documents}) Topics: {top_words}")

display_topics(hierarchical_topics)

Cluster 0 (Documents: 1129) Topics: tax, income, gst, sir, 2020, 39, pan, refund, rs, department
Cluster 0.0 (Documents: 207) Topics: tax, income, gst, sir, 2020, department, 39, refund, request, pan
Cluster 0.1 (Documents: 43) Topics: tax, gst, rs, income, sir, india, refund, 2021, demand, pan
Cluster 1 (Documents: 54) Topics: tax, gst, sir, 39, income, department, years, refund, request, government


In [17]:


import numpy as np
import pandas as pd

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based on the conditions
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get indices of filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data
    obtained_embeddings = np.array(stored_embeddings)[valid_indices]

    # Retrieve descriptions for the filtered data
    descriptions = filtered_data.loc[valid_indices, 'description'].tolist()
    filtered_data = filtered_data.loc[valid_indices]

    return obtained_embeddings, descriptions, filtered_data

# Inputs from user
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
initforward_str = input("Enter the initforward string: ")


data = df[df['initforward'].isna() == False]  # Filter data where 'initforward' is not NaN

# Get the embeddings and descriptions
obtained_embeddings, descriptions, filtered_data = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings)


def get_top_words(documents, n_top_words=10):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]
def recursive_clustering(level, data, embeddings, min_cluster_size=15, min_samples=None, results=None):
    if results is None:
        results = {}

    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, prediction_data=True)
    clusters = clusterer.fit_predict(embeddings)

    unique_clusters = set(clusters) - {-1}  # Exclude noise if present
    #unique_clusters = set(clusters)

    for cluster_id in unique_clusters:
        cluster_mask = clusters == cluster_id
        cluster_data = data.iloc[cluster_mask]
        cluster_embeddings = embeddings[cluster_mask]

        # Extract top words for the current cluster
        documents = cluster_data['description'].tolist()
        top_words = get_top_words(documents)

        # Store top words, their frequencies, and the number of documents in the cluster
        cluster_label = f"{level}{cluster_id}" if level else str(cluster_id)
        results[cluster_label] = {
            'top_words': [word for word, _ in top_words],
            'no_of_documents': len(cluster_data)
        }

        # Recursive call if the cluster is large enough
        if len(cluster_data) > 100:
            new_level = f"{cluster_label}."
            recursive_clustering(new_level, cluster_data, cluster_embeddings, min_cluster_size, min_samples, results)

    return results

hierarchical_topics = recursive_clustering("", filtered_data, obtained_embeddings)



def display_topics(hierarchical_topics, level=0):
    for cluster_label, info in hierarchical_topics.items():
        indent = '  ' * level
        top_words = ', '.join(info['top_words'])
        no_of_documents = info['no_of_documents']
        print(f"{indent}Cluster {cluster_label} (Documents: {no_of_documents}) Topics: {top_words}")

display_topics(hierarchical_topics)

Enter the starting date (YYYY-MM-DD):  2021-01-01
Enter the ending date (YYYY-MM-DD):  2021-12-31
Enter the initforward string:  1. DEABD - Department of Financial Services (Banking Division)


Cluster 0 (Documents: 51) Topics: bank, sir, loan, account, branch, 2021, rs, 160, help, hai
Cluster 1 (Documents: 20) Topics: bank, loan, sir, 2020, help, pay, senior, india, finance, request
Cluster 2 (Documents: 19) Topics: bank, 160, loan, 2020, branch, rs, passbook, 39, sir, entry
Cluster 3 (Documents: 28) Topics: bank, sir, 2020, 160, loan, rs, salary, request, account, 01
Cluster 4 (Documents: 22) Topics: bank, rs, loan, account, sir, 2020, hai, request, help, salary
Cluster 5 (Documents: 20) Topics: bank, loan, card, sir, request, account, credit, money, number, pension
Cluster 6 (Documents: 21) Topics: bank, branch, account, 39, sir, loan, credit, request, rs, customers
Cluster 7 (Documents: 2365) Topics: bank, loan, sir, account, branch, 39, rs, 2020, sbi, request
Cluster 7.0 (Documents: 19) Topics: bank, loan, sir, 39, branch, account, 2020, business, card, project
Cluster 7.1 (Documents: 16) Topics: bank, sir, 39, pay, hai, loan, help, ki, pension, march
Cluster 7.2 (Docume

In [19]:


import numpy as np
import pandas as pd

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based on the conditions
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get indices of filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data
    obtained_embeddings = np.array(stored_embeddings)[valid_indices]

    # Retrieve descriptions for the filtered data
    descriptions = filtered_data.loc[valid_indices, 'description'].tolist()
    filtered_data = filtered_data.loc[valid_indices]

    return obtained_embeddings, descriptions, filtered_data

# Inputs from user
starting_date = input("Enter the starting date (YYYY-MM-DD): ")
ending_date = input("Enter the ending date (YYYY-MM-DD): ")
initforward_str = input("Enter the initforward string: ")


data = df[df['initforward'].isna() == False]  # Filter data where 'initforward' is not NaN

# Get the embeddings and descriptions
obtained_embeddings, descriptions, filtered_data = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, stored_embeddings)


from sklearn.feature_extraction.text import TfidfVectorizer

def get_top_words_per_cluster(documents_by_cluster, n_top_words=10):
    # Check if documents_by_cluster contains any valid text
    if not any(len(doc.strip()) > 0 for doc in documents_by_cluster):
        return [[] for _ in documents_by_cluster]
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents_by_cluster)
    
    top_words_per_cluster = []
    for cluster_idx in range(tfidf_matrix.shape[0]):
        feature_array = np.array(tfidf_vectorizer.get_feature_names_out())
        tfidf_scores = tfidf_matrix[cluster_idx].toarray().flatten()
        top_indices = tfidf_scores.argsort()[-n_top_words:][::-1]
        top_words = feature_array[top_indices]
        top_scores = tfidf_scores[top_indices]
        top_words_per_cluster.append(list(zip(top_words, top_scores)))
    
    return top_words_per_cluster


def recursive_clustering(level, data, embeddings, min_cluster_size=15, min_samples=None, results=None):
    if results is None:
        results = {}

    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, prediction_data=True)
    clusters = clusterer.fit_predict(embeddings)

    unique_clusters = set(clusters) - {-1}  # Exclude noise if present

    # Prepare documents by cluster for class-based TF-IDF
    documents_by_cluster = [' '.join(data.iloc[clusters == cluster_id]['description'].tolist()) for cluster_id in unique_clusters]
    top_words_per_cluster = get_top_words_per_cluster(documents_by_cluster)

    for i, cluster_id in enumerate(unique_clusters):
        cluster_label = f"{level}{cluster_id}" if level else str(cluster_id)
        cluster_mask = clusters == cluster_id
        cluster_data = data.iloc[cluster_mask]

        # Store top words, their frequencies, and the number of documents in the cluster
        results[cluster_label] = {
            'top_words': [word for word, _ in top_words_per_cluster[i]],
            'no_of_documents': len(cluster_data)
        }

        # Recursive call if the cluster is large enough
        if len(cluster_data) > 100:
            new_level = f"{cluster_label}."
            recursive_clustering(new_level, cluster_data, embeddings[cluster_mask], min_cluster_size, min_samples, results)

    return results

hierarchical_topics = recursive_clustering("", filtered_data, obtained_embeddings)



def display_topics(hierarchical_topics, level=0):
    for cluster_label, info in hierarchical_topics.items():
        indent = '  ' * level
        top_words = ', '.join(info['top_words'])
        no_of_documents = info['no_of_documents']
        print(f"{indent}Cluster {cluster_label} (Documents: {no_of_documents}) Topics: {top_words}")

display_topics(hierarchical_topics)

Enter the starting date (YYYY-MM-DD):  2021-01-01
Enter the ending date (YYYY-MM-DD):  2021-12-31
Enter the initforward string:  1. DEABD - Department of Financial Services (Banking Division)


Cluster 0 (Documents: 51) Topics: bank, sir, loan, 160, account, branch, 2021, rs, hai, help
Cluster 1 (Documents: 20) Topics: bank, loan, sir, saab, help, senior, 2020, pay, citizens, finance
Cluster 2 (Documents: 19) Topics: bank, 160, passbook, loan, 2020, rs, branch, entry, 39, sir
Cluster 3 (Documents: 28) Topics: bank, 160, 2020, sir, loan, salary, rs, salaries, 01, request
Cluster 4 (Documents: 22) Topics: bank, rs, loan, account, sir, hai, 2020, request, salary, given
Cluster 5 (Documents: 20) Topics: bank, loan, card, idbi, request, sir, pension, rajesh, pai, account
Cluster 6 (Documents: 21) Topics: bank, fd, branch, account, sir, 39, loan, credit, customers, holders
Cluster 7 (Documents: 2365) Topics: bank, loan, sir, account, branch, 39, 160, rs, 2020, sbi
Cluster 7.0 (Documents: 19) Topics: bank, loan, 39, sir, branch, project, account, quot, card, 2020
Cluster 7.1 (Documents: 16) Topics: bank, sir, building, meri, pay, hai, 39, march, pension, loan
Cluster 7.2 (Documents:

In [156]:

topic_model = BERTopic()
docs = descriptions

topics, _ = topic_model.fit_transform(docs)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(11)  # Get 11 as one topic might be -1 (outliers)

# Print the top 10 topics and their top 10 words
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        print(f"Topic {topic}: ", ", ".join(words))

    

Topic 0:  hai, se, ki, ke, ka, ko, nahi, mai, mera, bhi
Topic 1:  subsidy, pmay, home, loan, housing, application, for, scheme, id, nhb
Topic 2:  account, branch, bank, of, the, is, to, open, not, address
Topic 3:  atm, transaction, amount, account, money, but, my, no, bank, debited
Topic 4:  mudra, business, loan, start, for, to, am, project, sir, my
Topic 5:  banks, privatisation, sector, public, are, private, banking, of, privatization, will
Topic 6:  pension, cppc, ppo, of, delhi, no, sbi, from, the, family
Topic 7:  loan, my, pay, family, am, help, me, job, to, amount
Topic 8:  pmegp, loan, project, under, the, for, application, manager, applied, unit
Topic 9:  education, 160, college, loan, my, university, fees, father, for, studies


## embedding using All-Mpnet-base-v2

In [21]:
from sentence_transformers import SentenceTransformer, models

model_name = 'all-mpnet-base-v2'  # This model is more powerful than MiniLM
embedding_model = SentenceTransformer(model_name)
embeddings = embedding_model.encode(docs, show_progress_bar=True, batch_size=32)

print("Embedding of the first document:", embeddings[0])
print("Shape of embeddings:", embeddings.shape)



.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/249 [00:00<?, ?it/s]

Embedding of the first document: [ 2.16074381e-02  1.19706634e-02  8.70135240e-03 -1.62722059e-02
 -1.23751964e-02 -2.16316953e-02 -2.41901837e-02  7.25022927e-02
 -7.08178245e-03  3.75240333e-02  7.39339832e-03  6.73705665e-03
  5.46423942e-02 -8.31892807e-03 -5.20825237e-02  2.64008921e-02
 -4.26560342e-02  2.56239846e-02 -1.37550719e-02 -2.77839135e-02
  1.80314842e-03  5.69313988e-02 -3.97609733e-02  1.34141222e-02
  1.54068591e-02  1.71681978e-02  1.94510762e-02 -6.68101460e-02
  1.18961586e-02  2.90017370e-02  4.31473320e-03 -1.45383719e-02
  1.20369429e-02 -3.20489854e-02  1.70769590e-06 -3.23383138e-02
  5.60662225e-02  1.61482058e-02 -7.08680507e-03  7.71600148e-03
 -2.58430596e-02 -5.13192154e-02 -3.84506620e-02  9.56505304e-04
 -1.06872180e-02  7.21483082e-02 -5.71417513e-05  8.16335827e-02
  2.42324620e-02  6.68060360e-03  3.94068658e-03 -2.39721723e-02
  5.73041253e-02 -4.52790409e-03 -3.49389575e-02 -6.59813806e-02
 -4.10370640e-02 -5.97784333e-02 -2.05336288e-02 -8.13885

## using hdbscan on Mpnet embeddings

In [22]:
import hdbscan

# Apply HDBSCAN 
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, prediction_data=True)
clusters = clusterer.fit_predict(embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]

# Assign cluster labels to the filtered clustered data
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

for cluster_num in set(clusters) - {-1}:  # Exclude noise (-1)
    # print(f"Documents in Cluster {cluster_num}:")
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    # print(cluster_docs.head())  # Display a few documents from the cluster
    
    # Get and print the top words for each cluster
    top_words = get_top_words(cluster_docs, 10)
    print(f"Topics in Cluster {cluster_num}: {', '.join([word for word, freq in top_words])}")
    print("\n")

Top words in Cluster 0: india, crypto, cryptocurrencies, 39, digital, currency, cryptocurrency, government, bitcoin, like


Top words in Cluster 1: ews, certificate, sbi, notification, 2019, 20, fy, valid, 2020, category


Top words in Cluster 2: bank, account, india, minor, years, age, state, opened, paheli, child


Top words in Cluster 3: industry, cashew, kerala, workers, banks, non, severe, crisis, years, processing


Top words in Cluster 4: idfc, bank, 2021, 21, bonds, maturity, money, matter, courier, date


Top words in Cluster 5: release, bhatia, non, maturity, proceeds, folio, ca, sameer, jatinder, pal


Top words in Cluster 6: bank, january, ramjee, singh, kaladumra, ps, nagar, siwan, 2008, retired


Top words in Cluster 7: bank, january, ramjee, singh, kaladumra, ps, nagar, siwan, 2008, retired


Top words in Cluster 8: prtpetition, fully, scanned, action, initiated, receipt, hard, copy, enclosures, sent


Top words in Cluster 9: rsmpetition, fully, scanned, action, initiate

## using umap and hdbscan on Mpnet embeddings

In [30]:

from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
reduced_embeddings = umap_model.fit_transform(embeddings)

import hdbscan



clusterer = hdbscan.HDBSCAN(min_cluster_size= 10 ,
                                                              metric='euclidean',
                                                              cluster_selection_method='eom',
                                                              prediction_data=True)
clusters = clusterer.fit_predict(reduced_embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]

# Assign cluster labels to the filtered clustered data
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

from sklearn.feature_extraction.text import CountVectorizer

def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

for cluster_num in set(clusters) - {-1}:  # Exclude noise (-1)
    # print(f"Documents in Cluster {cluster_num}:")
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    # print(cluster_docs.head())  # Display a few documents from the cluster
    
    # Get and print the top words for each cluster
    top_words = get_top_words(cluster_docs, 10)
    print(f"Top words in Cluster {cluster_num}: {', '.join([word for word, freq in top_words])}")
    print("\n")

Top words in Cluster 0: industry, cashew, kerala, banks, workers, processing, non, request, taken, severe


Top words in Cluster 1: bank, salary, salaries, paid, settlement, help, month, 2020, 01, arrears


Top words in Cluster 2: bank, salary, arrears, paid, 01, settlement, iba, board, 2021, future


Top words in Cluster 3: pg, fully, scanned, action, initiated, receipt, hard, copy, enclosures, sent


Top words in Cluster 4: bank, salary, salaries, settlement, paid, help, iba, future, arrears, work


Top words in Cluster 5: fully, scanned, action, initiated, receipt, hard, copy, enclosures, sent, shortly


Top words in Cluster 6: bank, salary, 01, salaries, paid, 2021, bipartite, settlement, 2020, india


Top words in Cluster 7: bank, salary, bipartite, board, india, employees, settlement, management, post, payments


Top words in Cluster 8: pg, rjspetition, fully, scanned, action, initiated, receipt, hard, copy, enclosures


Top words in Cluster 9: bank, january, ramjee, singh, kalad

In [25]:
import hdbscan
from sklearn.feature_extraction.text import TfidfVectorizer

def get_top_words_per_cluster(documents_by_cluster, n_top_words=10):
    # Check if documents_by_cluster contains any valid text
    if not any(len(doc.strip()) > 0 for doc in documents_by_cluster):
        return [[] for _ in documents_by_cluster]
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents_by_cluster)
    
    top_words_per_cluster = []
    for cluster_idx in range(tfidf_matrix.shape[0]):
        feature_array = np.array(tfidf_vectorizer.get_feature_names_out())
        tfidf_scores = tfidf_matrix[cluster_idx].toarray().flatten()
        top_indices = tfidf_scores.argsort()[-n_top_words:][::-1]
        top_words = feature_array[top_indices]
        top_scores = tfidf_scores[top_indices]
        top_words_per_cluster.append(list(zip(top_words, top_scores)))
    
    return top_words_per_cluster


def recursive_clustering(level, data, embeddings, min_cluster_size=15, min_samples=None, results=None):
    if results is None:
        results = {}

    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, prediction_data=True)
    clusters = clusterer.fit_predict(embeddings)

    unique_clusters = set(clusters) - {-1}  # Exclude noise if present

    # Prepare documents by cluster for class-based TF-IDF
    documents_by_cluster = [' '.join(data.iloc[clusters == cluster_id]['description'].tolist()) for cluster_id in unique_clusters]
    top_words_per_cluster = get_top_words_per_cluster(documents_by_cluster)

    for i, cluster_id in enumerate(unique_clusters):
        cluster_label = f"{level}{cluster_id}" if level else str(cluster_id)
        cluster_mask = clusters == cluster_id
        cluster_data = data.iloc[cluster_mask]

        # Store top words, their frequencies, and the number of documents in the cluster
        results[cluster_label] = {
            'top_words': [word for word, _ in top_words_per_cluster[i]],
            'no_of_documents': len(cluster_data)
        }

        # Recursive call if the cluster is large enough
        if len(cluster_data) > 100:
            new_level = f"{cluster_label}."
            recursive_clustering(new_level, cluster_data, embeddings[cluster_mask], min_cluster_size, min_samples, results)

    return results

hierarchical_topics = recursive_clustering("", filtered_data, embeddings)



def display_topics(hierarchical_topics, level=0):
    for cluster_label, info in hierarchical_topics.items():
        indent = '  ' * level
        top_words = ', '.join(info['top_words'])
        no_of_documents = info['no_of_documents']
        print(f"{indent}Cluster {cluster_label} (Documents: {no_of_documents}) Topics: {top_words}")

display_topics(hierarchical_topics)



Cluster 0 (Documents: 33) Topics: scanned, enclosures, petition, shortly, pg, fully, receipt, copy, sent, initiated
Cluster 1 (Documents: 44) Topics: scanned, enclosures, shortly, pg, fully, copy, receipt, sent, hard, initiated
Cluster 2 (Documents: 34) Topics: cashew, industry, workers, kerala, entrepreneurs, physical, factories, dwelling, coercive, collaterals
Cluster 3 (Documents: 24) Topics: rs, charges, refund, moratorium, 156, period, 2020, 08, gesture, suffering
Cluster 4 (Documents: 15) Topics: rrb, vacancies, baroda, ibps, withdraw, bob, exam, bank, examination, students
Cluster 5 (Documents: 16) Topics: oral, representation, applicant, details, received, gap, nb, complaints, नह, follows
Cluster 6 (Documents: 28) Topics: dhfl, fd, holders, ibc, resolution, rbi, public, lenders, 160, nhb
Cluster 7 (Documents: 74) Topics: banks, privatisation, public, bank, sector, private, psbs, privatization, psu, employees
Cluster 8 (Documents: 21) Topics: loan, salp, tonmay, bank, problem, k

## using hdbscan on Mpnet embeddings. Merge similar topics using connected components in graph

In [36]:
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix


# Apply HDBSCAN to the obtained embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, prediction_data=True)
clusters = clusterer.fit_predict(embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

# Define function to get top words
def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

# Extract top words for each cluster
cluster_top_words = []
for cluster_num in set(clusters) - {-1}:
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    top_words = get_top_words(cluster_docs, 10)
    top_words_str = ', '.join([word for word, freq in top_words])
    cluster_top_words.append(top_words_str)

# Generate TF-IDF matrix for top words of each cluster
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_top_words)

# Compute cosine similarity between cluster top words
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Identify clusters to merge by finding connected components in the similarity graph
similarity_threshold = 0.6
graph = cosine_sim_matrix > similarity_threshold
n_components, labels = connected_components(csgraph=csr_matrix(graph), directed=False)

# Display top words from merged clusters and remaining non-merged clusters
for component in range(n_components):
    # Find indices of clusters in the current component
    indices = np.where(labels == component)[0]
    if len(indices) == 1:
        # Non-merged cluster
        print(f"Non-merged Cluster {indices[0]} Topics: {cluster_top_words[indices[0]]}")
    else:
        # Merged clusters - displaying top words from one of the merging clusters arbitrarily
        print(f"Merged Clusters {indices.tolist()} Topics (from one of them): {cluster_top_words[indices[0]]}")


Non-merged Cluster 0 Topics: india, crypto, cryptocurrencies, 39, digital, currency, cryptocurrency, government, bitcoin, like
Non-merged Cluster 1 Topics: ews, certificate, sbi, notification, 2019, 20, fy, valid, 2020, category
Non-merged Cluster 2 Topics: bank, account, india, minor, years, age, state, opened, paheli, child
Non-merged Cluster 3 Topics: industry, cashew, kerala, workers, banks, non, severe, crisis, years, processing
Non-merged Cluster 4 Topics: idfc, bank, 2021, 21, bonds, maturity, money, matter, courier, date
Non-merged Cluster 5 Topics: release, bhatia, non, maturity, proceeds, folio, ca, sameer, jatinder, pal
Merged Clusters [6, 7] Topics (from one of them): bank, january, ramjee, singh, kaladumra, ps, nagar, siwan, 2008, retired
Merged Clusters [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] Topics (from one of them): prtpetition, fully, scanned, action, initiated, receipt, hard, copy, enclosures, sent
Non-merged Cluster 20 Topics: bank, money, moratorium, kapole,

In [46]:
import time
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix


t1 = time.time()
# Apply HDBSCAN to the obtained embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, prediction_data=True)
clusters = clusterer.fit_predict(embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

# Define function to get top words
def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

# Extract top words for each cluster
cluster_top_words = []
for cluster_num in set(clusters) - {-1}:
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    top_words = get_top_words(cluster_docs, 10)
    top_words_str = ', '.join([word for word, freq in top_words])
    cluster_top_words.append(top_words_str)

# Generate TF-IDF matrix for top words of each cluster
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_top_words)

# Compute cosine similarity between cluster top words
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Identify clusters to merge by finding connected components in the similarity graph
similarity_threshold = 0.6
graph = cosine_sim_matrix > similarity_threshold
n_components, labels = connected_components(csgraph=csr_matrix(graph), directed=False)

# Display top words from merged clusters and remaining non-merged clusters
for component in range(n_components):
    # Find indices of clusters in the current component
    indices = np.where(labels == component)[0]
    if len(indices) == 1:
        # Non-merged cluster
        print(f"Non-merged Cluster {indices[0]} Topics: {cluster_top_words[indices[0]]}")
    else:
        # Merged clusters - displaying top words from one of the merging clusters arbitrarily
        print(f"Merged Clusters {indices.tolist()} Topics (from one of them): {cluster_top_words[indices[0]]}")

t2 = time.time()
print("TIME: ", t2-t1)


Non-merged Cluster 0 Topics: india, crypto, cryptocurrencies, 39, digital, currency, cryptocurrency, government, bitcoin, like
Non-merged Cluster 1 Topics: ews, certificate, sbi, notification, 2019, 20, fy, valid, 2020, category
Non-merged Cluster 2 Topics: bank, account, india, minor, years, age, state, opened, paheli, child
Non-merged Cluster 3 Topics: industry, cashew, kerala, workers, banks, non, severe, crisis, years, processing
Non-merged Cluster 4 Topics: idfc, bank, 2021, 21, bonds, maturity, money, matter, courier, date
Non-merged Cluster 5 Topics: release, bhatia, non, maturity, proceeds, folio, ca, sameer, jatinder, pal
Merged Clusters [6, 7] Topics (from one of them): bank, january, ramjee, singh, kaladumra, ps, nagar, siwan, 2008, retired
Merged Clusters [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19] Topics (from one of them): prtpetition, fully, scanned, action, initiated, receipt, hard, copy, enclosures, sent
Non-merged Cluster 20 Topics: bank, money, moratorium, kapole,

## Increasing the cluster size

In [44]:
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix


# Apply HDBSCAN to the obtained embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size=15, prediction_data=True)
clusters = clusterer.fit_predict(embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

# Define function to get top words
def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

# Extract top words for each cluster
cluster_top_words = []
for cluster_num in set(clusters) - {-1}:
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    top_words = get_top_words(cluster_docs, 10)
    top_words_str = ', '.join([word for word, freq in top_words])
    cluster_top_words.append(top_words_str)

# Generate TF-IDF matrix for top words of each cluster
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_top_words)

# Compute cosine similarity between cluster top words
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Identify clusters to merge by finding connected components in the similarity graph
similarity_threshold = 0.6
graph = cosine_sim_matrix > similarity_threshold
n_components, labels = connected_components(csgraph=csr_matrix(graph), directed=False)

# Display top words from merged clusters and remaining non-merged clusters
for component in range(n_components):
    # Find indices of clusters in the current component
    indices = np.where(labels == component)[0]
    if len(indices) == 1:
        # Non-merged cluster
        print(f"Non-merged Cluster {indices[0]} Topics: {cluster_top_words[indices[0]]}")
    else:
        # Merged clusters - displaying top words from one of the merging clusters arbitrarily
        print(f"Merged Clusters {indices.tolist()} Topics (from one of them): {cluster_top_words[indices[0]]}")


Merged Clusters [0, 1] Topics (from one of them): pg, petition, fully, scanned, action, initiated, receipt, hard, copy, enclosures
Non-merged Cluster 2 Topics: industry, cashew, kerala, workers, banks, non, severe, crisis, years, processing
Non-merged Cluster 3 Topics: rs, charges, 2020, refund, moratorium, bank, 19, period, covid, 08
Non-merged Cluster 4 Topics: bank, rrb, vacancies, baroda, sir, ibps, bob, withdraw, examination, exam
Non-merged Cluster 5 Topics: oral, representation, received, details, applicant, pmo, 2021, nb, complaints, phone
Non-merged Cluster 6 Topics: dhfl, fd, holders, public, ibc, process, resolution, rbi, 160, nhb
Non-merged Cluster 7 Topics: banks, bank, public, sector, privatisation, 39, sir, private, india, employees
Non-merged Cluster 8 Topics: bank, loan, problem, kumar, tonmay, salp, sir, kindly, matter, 2021
Non-merged Cluster 9 Topics: bank, loan, idbi, money, request, rbi, application, charges, home, officer
Merged Clusters [10, 11, 12] Topics (from

## Bertopic on mpnet embeddings

In [49]:



topic_model = BERTopic()
docs = descriptions
doc_id = list(range(1, 7959))
# Fit the BERTopic model on the dummy documents and obtained embeddings
t1 = time.time()
topics, _ = topic_model.fit_transform(docs, embeddings)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(26)  # Get 11 as one topic might be -1 (outliers)

topic_dict = {}
# Print the top 25 topics and their top 10 words
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        #print(f"Topic {topic}: ", ", ".join(words))
        topic_dict[topic] = words
        
print(topic_dict)

{0: ['subsidy', 'pmay', 'home', 'loan', 'housing', 'scheme', 'application', 'have', 'for', 'from'], 1: ['hai', 'ki', 'se', 'ke', 'ka', 'ko', 'nahi', 'kar', 'bhi', 'aur'], 2: ['account', 'transaction', 'atm', 'amount', 'money', 'bank', 'rs', 'my', 'on', 'no'], 3: ['education', 'loan', 'student', 'college', 'my', 'university', 'applied', 'for', 'fees', 'studies'], 4: ['pmegp', 'loan', 'project', 'unit', 'msme', 'manager', 'for', 'under', 'the', 'scheme'], 5: ['banks', 'privatisation', 'sector', 'public', 'privatization', 'private', 'psu', 'psbs', 'employees', 'govt'], 6: ['branch', 'manager', 'account', 'the', 'of', 'bank', 'he', 'to', 'she', 'and'], 7: ['mudra', 'loan', 'business', 'start', 'for', 'sir', 'manager', 'my', 'have', 'am'], 8: ['business', 'we', 'for', 'loan', 'to', 'start', 'small', 'help', 'am', 'my'], 9: ['pension', 'ppo', 'cppc', 'family', 'no', 'pensioner', 'branch', 'my', 'railway', 'of'], 10: ['hdfc', 'card', 'credit', 'account', 'my', 'bank', 'and', 'payment', 'they'

In [59]:



topic_model = BERTopic()
docs = descriptions
doc_id = list(range(1, 7959))  # List of registration Numbers.....

topics, _ = topic_model.fit_transform(docs, embeddings)


# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

# Get the top 10 topics
top_topics = topic_info.head(26)  # Get 26 as one topic might be -1 (outliers)

topic_dict = {}
doc_topic_dict = {}  # Initialize a new dictionary to store document IDs for each topic

# Populate doc_topic_dict with document IDs for each topic
for doc_index, topic_number in enumerate(topics):
    if topic_number != -1:  # excluding the outlier cluster
        if topic_number in doc_topic_dict:
            doc_topic_dict[topic_number].append(doc_id[doc_index])
        else:
            doc_topic_dict[topic_number] = [doc_id[doc_index]]

# Iterate through the top topics and add document IDs to topic_dict
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Combine words and document IDs
        topic_info = {
            'words': words,
            'doc_ids': doc_topic_dict.get(topic, [])
        }
        topic_dict[topic] = topic_info

print(topic_dict)


{0: {'words': ['subsidy', 'pmay', 'home', 'loan', 'housing', 'scheme', 'application', 'have', 'for', 'from'], 'doc_ids': [9, 33, 38, 53, 63, 84, 85, 87, 92, 114, 119, 122, 136, 151, 154, 161, 202, 217, 218, 220, 226, 231, 245, 247, 257, 275, 283, 288, 301, 303, 308, 311, 313, 323, 328, 334, 348, 368, 380, 388, 396, 398, 407, 414, 442, 465, 482, 492, 504, 506, 535, 567, 604, 625, 664, 670, 685, 702, 708, 727, 738, 742, 749, 750, 765, 768, 780, 794, 797, 807, 809, 845, 854, 891, 914, 918, 922, 930, 941, 944, 945, 957, 970, 973, 992, 1004, 1028, 1057, 1063, 1072, 1091, 1119, 1121, 1143, 1152, 1165, 1176, 1186, 1189, 1202, 1209, 1216, 1217, 1220, 1224, 1237, 1238, 1254, 1281, 1282, 1351, 1382, 1404, 1410, 1421, 1453, 1457, 1474, 1477, 1489, 1502, 1517, 1523, 1527, 1528, 1531, 1540, 1549, 1583, 1597, 1601, 1602, 1603, 1618, 1622, 1626, 1634, 1639, 1659, 1667, 1687, 1708, 1716, 1739, 1754, 1756, 1761, 1766, 1770, 1772, 1773, 1777, 1786, 1803, 1822, 1833, 1837, 1842, 1862, 1864, 1873, 1888, 1

In [ ]:
from bertopic import BERTopic
import numpy as np
import pandas as pd

def get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, embeddings):
    # Convert the 'regdate' column to datetime
    data['regdate'] = pd.to_datetime(data['regdate'])

    # Filter the data based on the conditions
    filtered_data = data[(data['regdate'] >= pd.to_datetime(starting_date)) &
                         (data['regdate'] <= pd.to_datetime(ending_date)) &
                         (data['initforward'] == initforward_str)]

    # Get indices of filtered data
    indices = filtered_data.index.tolist()

    # Ensure indices are within the range of stored_embeddings length
    valid_indices = [index for index in indices if index < len(stored_embeddings)]

    # Retrieve embeddings for the filtered data
    obtained_embeddings = np.array(stored_embeddings)[valid_indices]

    # Retrieve descriptions for the filtered data
    descriptions = filtered_data.loc[valid_indices, 'description'].tolist()
    filtered_data = filtered_data.loc[valid_indices]

    return obtained_embeddings, descriptions



def custom_rca():
    # Inputs from user
    starting_date = input("Enter the starting date (YYYY-MM-DD): ")
    ending_date = input("Enter the ending date (YYYY-MM-DD): ")
    initforward_str = input("Enter the initforward string: ")
    data = df[df['initforward'].isna() == False]  # Filter data where 'initforward' is not NaN
    # Get the embeddings and descriptions
    obtained_embeddings, descriptions = get_embeddings_by_date_and_initforward(starting_date, ending_date, initforward_str, data, embeddings)
    
    topic_model = BERTopic()
    docs = descriptions
    doc_id = list(range(1, 7959))  # List of registration Numbers.....

    
    # Fit the BERTopic model on the dummy documents and obtained embeddings
    
    topics, _ = topic_model.fit_transform(docs, obtained_embeddings)
    
    # Extract the topics and their frequencies
    topic_info = topic_model.get_topic_info()
    
    # Get the top 25 topics
    top_topics = topic_info.head(26)  # Get 26 as one topic might be -1 (outliers)
    
    topic_dict = {}
    doc_topic_dict = {}  # Initialize a new dictionary to store document IDs for each topic
    
    # Populate doc_topic_dict with document IDs for each topic
    for doc_index, topic_number in enumerate(topics):
        if topic_number != -1:  # excluding the outlier cluster
            if topic_number in doc_topic_dict:
                doc_topic_dict[topic_number].append(doc_id[doc_index])
            else:
                doc_topic_dict[topic_number] = [doc_id[doc_index]]
    
    # Iterate through the top topics and add document IDs to topic_dict
    for topic in top_topics['Topic']:
        if topic != -1:  # excluding the outlier cluster
            # Get top 10 words for the topic
            topic_words = topic_model.get_topic(topic)[:10]
            # Extract only the words (not the scores)
            words = [word for word, _ in topic_words]
            # Combine words and document IDs
            topic_info = {
                'words': words,
                'doc_ids': doc_topic_dict.get(topic, [])
            }
            topic_dict[topic] = topic_info

    return topic_dict

results = custom_rca()


# Berttopic on documents

In [ ]:
topic_model = BERTopic()
docs = descriptions
# Fit the BERTopic model on the dummy documents and obtained embeddings
topics, _ = topic_model.fit_transform(docs)

# Extract the topics and their frequencies
topic_info = topic_model.get_topic_info()

In [42]:
# Get the top 10 topics
top_topics = topic_info.head(26)  # Get 11 as one topic might be -1 (outliers)

# Print the top 10 topics and their top 10 words
for topic in top_topics['Topic']:
    if topic != -1:  # excluding the outlier cluster
        # Get top 10 words for the topic
        topic_words = topic_model.get_topic(topic)[:10]
        # Extract only the words (not the scores)
        words = [word for word, _ in topic_words]
        # Print the topic and its words
        print(f"Topic {topic}: ", ", ".join(words))

Topic 0:  hai, se, ki, ke, ka, ko, nahi, mai, mera, bhi
Topic 1:  atm, account, transaction, money, amount, my, bank, the, but, no
Topic 2:  pension, cppc, of, the, pensioners, ppo, to, no, sbi, from
Topic 3:  subsidy, pmay, home, application, id, loan, housing, have, my, from
Topic 4:  banks, privatisation, sector, public, private, are, privatization, banking, employees, of
Topic 5:  mudra, business, loan, start, for, project, to, sir, my, am
Topic 6:  my, loan, pay, family, am, help, me, to, now, amount
Topic 7:  pmegp, loan, project, under, rejected, for, the, applied, application, manager
Topic 8:  hdfc, card, credit, restructure, bank, charges, dated, complaint, and, my
Topic 9:  cheque, account, book, cheques, bank, of, number, hdfc, the, branch
Topic 10:  card, sbi, credit, cards, charges, payment, to, they, have, call
Topic 11:  dhfl, fd, holders, ibc, resolution, public, process, invested, nhb, fds
Topic 12:  transfer, my, in, daughter, he, her, family, is, of, to
Topic 13:  u

In [39]:
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix


from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=384, min_dist=0.0, metric='cosine')
reduced_embeddings = umap_model.fit_transform(embeddings)


# Apply HDBSCAN to the obtained embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size= 10 ,
                                                              metric='euclidean',
                                                              cluster_selection_method='eom',
                                                              prediction_data=True)
clusters = clusterer.fit_predict(reduced_embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

# Define function to get top words
def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

# Extract top words for each cluster
cluster_top_words = []
for cluster_num in set(clusters) - {-1}:
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    top_words = get_top_words(cluster_docs, 10)
    top_words_str = ', '.join([word for word, freq in top_words])
    cluster_top_words.append(top_words_str)

# Generate TF-IDF matrix for top words of each cluster
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_top_words)

# Compute cosine similarity between cluster top words
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Identify clusters to merge by finding connected components in the similarity graph
similarity_threshold = 0.6
graph = cosine_sim_matrix > similarity_threshold
n_components, labels = connected_components(csgraph=csr_matrix(graph), directed=False)

# Display top words from merged clusters and remaining non-merged clusters
for component in range(n_components):
    # Find indices of clusters in the current component
    indices = np.where(labels == component)[0]
    if len(indices) == 1:
        # Non-merged cluster
        print(f"Non-merged Cluster {indices[0]} Topics: {cluster_top_words[indices[0]]}")
    else:
        # Merged clusters - displaying top words from one of the merging clusters arbitrarily
        print(f"Merged Clusters {indices.tolist()} Topics (from one of them): {cluster_top_words[indices[0]]}")


Non-merged Cluster 0 Topics: industry, cashew, kerala, banks, workers, non, severe, crisis, years, processing
Merged Clusters [1, 2, 4, 5, 9, 19, 20] Topics (from one of them): fully, scanned, action, initiated, receipt, hard, copy, enclosures, sent, shortly
Merged Clusters [3, 6, 7, 11] Topics (from one of them): bank, salary, salaries, paid, settlement, help, month, 2020, 01, arrears
Merged Clusters [8, 10] Topics (from one of them): bank, loan, problem, salp, kumar, sir, tonmay, emi, hdfc, 2021
Non-merged Cluster 12 Topics: bank, salary, bipartite, board, management, employees, india, 11th, post, payments
Non-merged Cluster 13 Topics: oral, representation, received, details, applicant, 2021, pmo, complaints, nb, phone
Non-merged Cluster 14 Topics: credit, rs, 000, card, situation, bank, waive, ceremony, help, 50
Non-merged Cluster 15 Topics: rs, charges, 2020, refund, bank, moratorium, 19, period, covid, 08
Non-merged Cluster 16 Topics: loan, subsidy, bank, pmay, home, sir, applicat

In [40]:
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix


from umap import UMAP

umap_model = UMAP(n_neighbors=45, n_components=384, min_dist=0.0, metric='cosine')
reduced_embeddings = umap_model.fit_transform(embeddings)


# Apply HDBSCAN to the obtained embeddings
clusterer = hdbscan.HDBSCAN(min_cluster_size= 10 ,
                                                              metric='euclidean',
                                                              cluster_selection_method='eom',
                                                              prediction_data=True)
clusters = clusterer.fit_predict(reduced_embeddings)

# Filter the original data to include only those rows that were clustered
clustered_indices = [i for i, label in enumerate(clusters) if label != -1]
filtered_clustered_data = filtered_data.iloc[clustered_indices]
filtered_clustered_data['Cluster'] = clusters[clusters != -1]

# Define function to get top words
def get_top_words(documents, n_top_words):
    vec = CountVectorizer(stop_words='english').fit(documents)
    bag_of_words = vec.transform(documents)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top_words]

# Extract top words for each cluster
cluster_top_words = []
for cluster_num in set(clusters) - {-1}:
    cluster_docs = filtered_clustered_data[filtered_clustered_data['Cluster'] == cluster_num]['description']
    top_words = get_top_words(cluster_docs, 10)
    top_words_str = ', '.join([word for word, freq in top_words])
    cluster_top_words.append(top_words_str)

# Generate TF-IDF matrix for top words of each cluster
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_top_words)

# Compute cosine similarity between cluster top words
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Identify clusters to merge by finding connected components in the similarity graph
similarity_threshold = 0.6
graph = cosine_sim_matrix > similarity_threshold
n_components, labels = connected_components(csgraph=csr_matrix(graph), directed=False)

# Display top words from merged clusters and remaining non-merged clusters
for component in range(n_components):
    # Find indices of clusters in the current component
    indices = np.where(labels == component)[0]
    if len(indices) == 1:
        # Non-merged cluster
        print(f"Non-merged Cluster {indices[0]} Topics: {cluster_top_words[indices[0]]}")
    else:
        # Merged clusters - displaying top words from one of the merging clusters arbitrarily
        print(f"Merged Clusters {indices.tolist()} Topics (from one of them): {cluster_top_words[indices[0]]}")


Non-merged Cluster 0 Topics: industry, cashew, kerala, banks, workers, processing, non, request, taken, severe
Non-merged Cluster 1 Topics: bank, january, ramjee, singh, kaladumra, ps, nagar, siwan, 2008, retired
Merged Clusters [2, 8] Topics (from one of them): bank, salary, 01, paid, arrears, 2021, settlement, iba, salaries, board
Merged Clusters [3, 22] Topics (from one of them): bank, loan, problem, salp, kumar, sir, tonmay, emi, hdfc, 2021
Non-merged Cluster 4 Topics: credit, rs, 000, card, situation, bank, waive, ceremony, help, 50
Non-merged Cluster 5 Topics: bank, salary, salaries, paid, settlement, help, month, sir, 2020, future
Non-merged Cluster 6 Topics: oral, representation, received, details, applicant, 2021, pmo, complaints, nb, phone
Non-merged Cluster 7 Topics: dhfl, fd, holders, public, ibc, process, rbi, resolution, money, 160
Non-merged Cluster 9 Topics: bank, salary, bipartite, india, board, employees, 2021, settlement, 11th, payments
Merged Clusters [10, 11, 12, 1




## FINAL EXPERIMENTS


In [ ]:



from sentence_transformers import SentenceTransformer, models

model_name = 'all-mpnet-base-v2'  # This model is more powerful than MiniLM
embedding_model = SentenceTransformer(model_name)
embeddings = embedding_model.encode(data['description'].tolist(), show_progress_bar=True, batch_size=32)

print("Embedding of the first document:", embeddings[0])
print("Shape of embeddings:", embeddings.shape)


In [48]:


import numpy as np
import pandas as pd




from sklearn.feature_extraction.text import TfidfVectorizer

def get_top_words_per_cluster(documents_by_cluster, n_top_words=10):
    # Check if documents_by_cluster contains any valid text
    if not any(len(doc.strip()) > 0 for doc in documents_by_cluster):
        return [[] for _ in documents_by_cluster]
    
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents_by_cluster)
    
    top_words_per_cluster = []
    for cluster_idx in range(tfidf_matrix.shape[0]):
        feature_array = np.array(tfidf_vectorizer.get_feature_names_out())
        tfidf_scores = tfidf_matrix[cluster_idx].toarray().flatten()
        top_indices = tfidf_scores.argsort()[-n_top_words:][::-1]
        top_words = feature_array[top_indices]
        top_scores = tfidf_scores[top_indices]
        top_words_per_cluster.append(list(zip(top_words, top_scores)))
    
    return top_words_per_cluster


def recursive_clustering(level, data, embeddings, min_cluster_size=15, min_samples=None, results=None):
    if results is None:
        results = {}

    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, prediction_data=True)
    clusters = clusterer.fit_predict(embeddings)

    unique_clusters = set(clusters) - {-1}  # Exclude noise if present

    # Prepare documents by cluster for class-based TF-IDF
    documents_by_cluster = [' '.join(data.iloc[clusters == cluster_id]['description'].tolist()) for cluster_id in unique_clusters]
    top_words_per_cluster = get_top_words_per_cluster(documents_by_cluster)

    for i, cluster_id in enumerate(unique_clusters):
        cluster_label = f"{level}{cluster_id}" if level else str(cluster_id)
        cluster_mask = clusters == cluster_id
        cluster_data = data.iloc[cluster_mask]

        # Store top words, their frequencies, and the number of documents in the cluster
        results[cluster_label] = {
            'top_words': [word for word, _ in top_words_per_cluster[i]],
            'no_of_documents': len(cluster_data)
        }

        # Recursive call if the cluster is large enough
        if len(cluster_data) > 100:
            new_level = f"{cluster_label}."
            recursive_clustering(new_level, cluster_data, embeddings[cluster_mask], min_cluster_size, min_samples, results)

    return results

hierarchical_topics = recursive_clustering("", filtered_data, embeddings)



def display_topics(hierarchical_topics, level=0):
    for cluster_label, info in hierarchical_topics.items():
        indent = '  ' * level
        top_words = ', '.join(info['top_words'])
        no_of_documents = info['no_of_documents']
        print(f"{indent}Cluster {cluster_label} (Documents: {no_of_documents}) Topics: {top_words}")

display_topics(hierarchical_topics)

Cluster 0 (Documents: 33) Topics: scanned, enclosures, petition, shortly, pg, fully, receipt, copy, sent, initiated
Cluster 1 (Documents: 44) Topics: scanned, enclosures, shortly, pg, fully, copy, receipt, sent, hard, initiated
Cluster 2 (Documents: 34) Topics: cashew, industry, workers, kerala, entrepreneurs, physical, factories, dwelling, coercive, collaterals
Cluster 3 (Documents: 24) Topics: rs, charges, refund, moratorium, 156, period, 2020, 08, gesture, suffering
Cluster 4 (Documents: 15) Topics: rrb, vacancies, baroda, ibps, withdraw, bob, exam, bank, examination, students
Cluster 5 (Documents: 16) Topics: oral, representation, applicant, details, received, gap, nb, complaints, नह, follows
Cluster 6 (Documents: 28) Topics: dhfl, fd, holders, ibc, resolution, rbi, public, lenders, 160, nhb
Cluster 7 (Documents: 74) Topics: banks, privatisation, public, bank, sector, private, psbs, privatization, psu, employees
Cluster 8 (Documents: 21) Topics: loan, salp, tonmay, bank, problem, k

In [50]:
type(embeddings)

numpy.ndarray

In [51]:
type(embeddings[0])

numpy.ndarray

In [52]:
embeddings.shape

(7958, 768)

In [58]:
# embeddings[0].shape
doc_id = list(range(1, 7959))
print(len(doc_id))

7958
